# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

In [100]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [118]:
cluster0 =[ 
'used_memory_rss',
'used_memory_rss_human',
'rss_overhead_ratio',
'rss_overhead_bytes',
'mem_fragmentation_ratio',
'mem_fragmentation_bytes']

cluster1 = [
    'uptime_in_seconds',
    'hz',
    'lru_clock',
    'used_memory_peak',
    'maxmemory',
    'maxmemory_human',
    'rdb_last_save_time',
    'total_net_output_bytes',
    'used_cpu_sys',
    'used_cpu_user'
]

cluster2 = [
    'allocator_allocated',
    'allocator_active',
    'allocator_frag_bytes',
    'instantaneous_ops_per_sec',
    'instantaneous_input_kbps',
    'instantaneous_output_kbps'
]

cluster3 = [
    'allocator_resident',
    'allocator_rss_ratio',
    'allocator_rss_bytes'  
]

knob_list = [
    "save0_0",
    "save0_1",
    "save1_0",
    "save1_1",
    "save2_0",
    "save2_1",

    "rdbcompression",
    "rdbchecksum",
    "rdb-save-incremental-fsync",

    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",
    
    "maxmemory-policy",
    "maxmemory-samples",
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del",

    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",
    "hz",
    "dynamic-hz"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [102]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

In [103]:
# 데이터 불러오기 knob_list
knobs = pd.read_csv('./data/result_config0.csv')

In [104]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

## 데이터 전처리

In [105]:
# 클러스터 합치기
cluster = cluster0+cluster1+cluster2+cluster3

In [106]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)
total_internal_df.head()

total_internal_df0 = extract_data(cluster0, internal_csv)
total_internal_df1 = extract_data(cluster1, internal_csv)
total_internal_df2 = extract_data(cluster2, internal_csv)
total_internal_df3 = extract_data(cluster3, internal_csv)

In [107]:
# throughput 결측치 제거
total_internal_df.dropna()

total_internal_df0.dropna()
total_internal_df1.dropna()
total_internal_df2.dropna()
total_internal_df3.dropna()

,allocator_resident,allocator_rss_ratio,allocator_rss_bytes
0,42209280,1.08,3276800
1,44310528,1.14,5378048
2,42209280,1.08,3276800
3,42209280,1.08,3276800
4,42209280,1.08,3276800
...,...,...,...
2495,42160128,1.08,3186688
2496,44310528,1.14,5378048
2497,42209280,1.08,3276800
2498,44310528,1.14,5378048


In [108]:
# maxmemory 처리 throughtput은 처리할 필요없다.
total_internal_df['maxmemory']

0       2831155200
1       2306867200
2       1782579200
3       1048576000
4       1992294400
           ...    
2495    2936012800
2496    1572864000
2497    3040870400
2498    2306867200
2499    1468006400
Name: maxmemory, Length: 2500, dtype: int64

In [109]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [110]:
# knob_list 만 추출, AOF 관련 제외
subTotal_knobs = extract_data(knob_list, knobs)

In [163]:
# activedefrag knobs 분리

# activedefrag O
subTotal_knobs_Active = subTotal_knobs.dropna()
Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# activedefrag X
subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# activedefrag O and X
subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
total_index = list(subTotal_knobs_T.index)   # 인덱스

In [241]:
# knobs 에서 object 형 찾기

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [171]:
# activedefrag O
total_internal_Active = total_internal_df.iloc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.iloc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.iloc[total_index]

## 상관분석

In [42]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


In [224]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)

In [174]:
# total_internal_s = [total_internal_df0, total_internal_df1,
#                    total_internal_df2, total_internal_df3]

# for i in total_internal_s:
#     df_t = cal_spearmanr(knobs['hash-max-ziplist-value'], i)
#     print(df_t)


## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [239]:
numericalActive_var
numericalActive_var_new = numericalActive_var[6:10]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [240]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
                         corr   p-value
allocator_resident   0.055844  0.048207
allocator_rss_ratio  0.057253  0.042821
allocator_rss_bytes  0.056016  0.047520

kendalltau
                        corr  p-value
rss_overhead_ratio -0.051988  0.02251

spearman
                        corr   p-value
rss_overhead_ratio -0.064179  0.023149

###active-defrag-threshold-upper###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-min###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-max###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns

In [315]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1, cluster2, cluster3]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().sum())

    active_result['cluster'+str(idx)] = cluster_val

In [316]:
active_result

,cluster0,cluster1,cluster2,cluster3
active-defrag-threshold-lower,0.107440,0.250392,0.081287,0.169114
active-defrag-threshold-upper,0.210770,0.155932,0.233161,0.055846
active-defrag-cycle-min,0.143515,0.214407,0.144289,0.089408
active-defrag-cycle-max,0.062975,0.182104,0.079199,0.024199


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [243]:
numerical_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [244]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                            corr   p-value
mem_fragmentation_ratio -0.03388  0.036831

spearman
                             corr   p-value
mem_fragmentation_ratio -0.041749  0.036861

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                             corr   p-value
mem_fragmentation_ratio  0.032439  0.045625
allocator_resident       0.037987  0.015417

spearman
                             corr   p-value
mem_fragmentation_ratio  0.040057  0.045217
allocator_resident       0.048404  0.015503

###save1_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [

In [318]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1, cluster2, cluster3]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().sum())

    numerical_result['cluster'+str(idx)] = cluster_val

In [319]:
numerical_result

,cluster0,cluster1,cluster2,cluster3
save0_0,0.131075,0.089273,0.087590,0.029786
save0_1,0.119660,0.148474,0.047737,0.054440
save1_0,0.120897,0.132394,0.062789,0.105810
save1_1,0.107582,0.087805,0.081727,0.042945
save2_0,0.067284,0.175357,0.083118,0.019064
save2_1,0.140120,0.222992,0.050866,0.016158
maxmemory,0.052251,1.805280,0.027295,0.007078
maxmemory-samples,0.037957,0.183298,0.065315,0.014346
hash-max-ziplist-entries,0.130086,0.068355,0.058503,0.054297
hash-max-ziplist-value,0.096841,0.121044,0.038630,0.085916


## other knob
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [245]:
numericalActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [246]:
extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
        corr   p-value
hz  0.037937  0.047294

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
                        corr   p-value
lru_clock           0.059351  0.035746
rdb_last_save_time  0.059352  0.035743

kendalltau
                           corr   p-value
used_memory_rss       -0.044470  0.030427
used_memory_rss_human -0.047318  0.027262
lru_clock              0.040269  0.042872
rdb_last_save_time     0.040269  0.042872

spearman
                           corr   p-value
used_memory_rss       -0.060946  0.031056
used_memory_rss_human -0.062604  0.026751
lru_clock              0.057114  0.043329
rdb_last_save_time     0.057114  0.043329

###save1_0###
pearson
                        corr   p-value
lru_clock          -0.070523  0.012561
rdb_last_save_time -0.070522  0.012563
used_cpu_sys       -0.061330  0.030011

kendalltau
                        corr   p-va

## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [247]:
numericalNoActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [249]:
extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_0###
pearson
                         corr   p-value
lru_clock            0.058587  0.038509
rdb_last_save_time   0.058689  0.038169
allocator_resident   0.059750  0.034810
allocator_rss_ratio  0.058408  0.039104
allocator_rss_bytes  0.059935  0.034250

kendalltau
                             corr   p-value
mem_fragmentation_ratio  0.051463  0.025128
lru_clock                0.039195  0.038311
rdb_last_save_time       0.039195  0.038311
allocator_resident       0.053478  0.014550
allocator_rss_ratio      0.046319  0.045418
allocator_rss_bytes      0.047804  0.024630

spearman
            

---

## pointbiserialr 분석

In [260]:
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()


# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [254]:
boolean_var

['rdbcompression',
 'rdbchecksum',
 'rdb-save-incremental-fsync',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del',
 'activerehashing',
 'dynamic-hz']

In [255]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [261]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                            corr   p-value
total_net_output_bytes -0.056501  0.004715

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
                         corr   p-value
rss_overhead_ratio  -0.046519  0.020017
rss_overhead_bytes  -0.048922  0.014431
allocator_resident   0.043003  0.031551
allocator_rss_ratio  0.043123  0.031080
allocator_rss_bytes  0.043044  0.031389

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
        corr   p-value
hz -0.045644  0.022474

###activerehashing###
pointbierialr
                         corr   p-value
allocator_allocated -0.044451  0.026248
allocator_active    -0.040469  0.043047

###dynamic-hz###
pointbierialr
                     corr   p-value
ma

In [313]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1, cluster2, cluster3]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().sum())

    bool_result['cluster'+str(idx)] = cluster_val

In [314]:
bool_result

,cluster0,cluster1,cluster2,cluster3
rdbcompression,0.043045,0.247608,0.132567,0.017334
rdbchecksum,0.033166,0.112314,0.049670,0.035439
rdb-save-incremental-fsync,0.134179,0.094039,0.083578,0.051800
lazyfree-lazy-eviction,0.186498,0.111021,0.024066,0.129169
lazyfree-lazy-expire,0.122251,0.127492,0.093513,0.023721
lazyfree-lazy-server-del,0.076782,0.175183,0.051966,0.071605
activerehashing,0.131495,0.178660,0.191599,0.079875
dynamic-hz,0.089357,0.175908,0.114635,0.082941


---

In [ ]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)